# 6. Generazione delle Visualizzazioni per la Presentazione

**Obiettivo:** Creare e salvare i grafici specifici progettati per la presentazione finale del progetto.

**Logica:** Ogni cella di codice è dedicata a generare una visualizzazione per una specifica slide.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import desc, col, avg

# Aggiunge la root del progetto al sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importazione delle utility
from src.utils.helpers import get_spark_session
from src.clustering.models import get_cluster_profiles
from src.config.spark_config import SPARK_CONFIG

# Inizializzazione della sessione Spark
spark = get_spark_session(
    app_name="NBA_Presentation_Visuals",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

# Impostazioni globali per i grafici
sns.set_theme(style="whitegrid")
FIG_OUTPUT_PATH = "../reports/figures"
os.makedirs(FIG_OUTPUT_PATH, exist_ok=True)
print(f"I grafici verranno salvati in: {os.path.abspath(FIG_OUTPUT_PATH)}")

### Caricamento dei Dati Necessari
Carichiamo sia il dataset con le metriche avanzate (per analisi pre-clustering) sia quello con i risultati del clustering.

In [ ]:
# Percorsi dei file Parquet
adv_metrics_path = "../data/processed/players_advanced_metrics.parquet"
clustered_path = "../data/processed/players_clustered.parquet"

# Caricamento dei DataFrame
df_adv_metrics = spark.read.parquet(adv_metrics_path)
df_clustered = spark.read.parquet(clustered_path)

### Grafico sull'Effetto della Normalizzazione

Mostriamo come la normalizzazione 'per 36 minuti' permetta un confronto più significativo tra un giocatore titolare (con alto minutaggio) e una riserva (con minutaggio inferiore), rivelandone il potenziale impatto.

In [ ]:
# Filtra per la stagione 2023
df_2023 = df_adv_metrics.filter(col('season') == 2023)

# Selezione di un titolare e una riserva per il confronto
starter = df_2023.orderBy(desc("mp")).first()
median_mp = df_2023.approxQuantile("mp", [0.5], 0.01)[0]
reserve = df_2023.filter(col('mp') >= median_mp).orderBy("mp").first()

# Preparazione dei dati per il grafico
plot_data = pd.DataFrame([
    {'player': starter['player'], 'type': 'Punti Totali', 'value': starter['pts']},
    {'player': starter['player'], 'type': 'Punti per 36 min', 'value': starter['pts_per_36_min']},
    {'player': reserve['player'], 'type': 'Punti Totali', 'value': reserve['pts']},
    {'player': reserve['player'], 'type': 'Punti per 36 min', 'value': reserve['pts_per_36_min']}
])

# Creazione del grafico a barre
plt.figure(figsize=(10, 6))
sns.barplot(data=plot_data, x='player', y='value', hue='type', palette=['#ff7f0e', '#1f77b4'])
plt.title('Effetto della Normalizzazione: Titolare vs. Riserva', fontsize=16)
plt.ylabel('Valore Statistico (Punti)')
plt.xlabel('Giocatore')
plt.legend(title='Tipo di Statistica')
plt.tight_layout()
plt.savefig(os.path.join(FIG_OUTPUT_PATH, 'slide_7_normalization_effect.png'))
plt.show()

### Grafico Top 10 Giocatori per True Shooting % (TS%)

Evidenziamo i giocatori più efficienti al tiro nella stagione 2023. Filtriamo per un numero minimo di tiri tentati (`fga > 250`) per escludere giocatori con campioni troppo piccoli.

In [ ]:
# Seleziona i top 10 giocatori per TS% con un volume di tiri significativo
top_ts_players = df_adv_metrics.filter((col('season') == 2023) & (col('fga') > 250)) \
                                .orderBy(desc("ts_pct_calc")) \
                                .limit(10) \
                                .toPandas()

# Creazione del grafico a barre orizzontale
plt.figure(figsize=(12, 8))
sns.barplot(data=top_ts_players, x='ts_pct_calc', y='player', palette='viridis')
plt.title('Top 10 Giocatori per Efficienza al Tiro (TS%) - Stagione 2023', fontsize=16)
plt.xlabel('True Shooting Percentage (TS%)')
plt.ylabel('Giocatore')
plt.tight_layout()
plt.savefig(os.path.join(FIG_OUTPUT_PATH, 'slide_8_top_ts_players.png'))
plt.show()

### Grafici di Confronto per Ciascun Cluster

Generiamo un grafico per ogni cluster, confrontando le sue statistiche chiave con la media generale della lega per evidenziarne le peculiarità.

In [ ]:
# 1. Calcolo delle medie generali per tutte le feature
feature_cols = [
    'pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min', 
    'stl_per_36_min', 'blk_per_36_min', 'tov_per_36_min',
    'ts_pct_calc'
]
overall_averages = df_clustered.select([avg(c).alias(c) for c in feature_cols]).collect()[0].asDict()

# 2. Calcolo dei profili medi per cluster
cluster_profiles_df = get_cluster_profiles(df_clustered, feature_cols).toPandas().set_index('cluster_id')

# 3. Funzione di plotting riutilizzabile
def plot_cluster_comparison(cluster_id, stats_to_plot, cluster_profiles, overall_avg, color, save_name, title):
    data_list = []
    for stat in stats_to_plot:
        clean_stat_name = stat.replace('_per_36_min', '').replace('_calc', '').upper()
        data_list.append({'stat': clean_stat_name, 'type': f'Cluster {cluster_id}', 'value': cluster_profiles.loc[cluster_id, f'avg_{stat}']})
        data_list.append({'stat': clean_stat_name, 'type': 'Media Generale', 'value': overall_avg[stat]})
    
    plot_df = pd.DataFrame(data_list)
    
    plt.figure(figsize=(8, 5))
    sns.barplot(data=plot_df, x='stat', y='value', hue='type', palette=[color, 'lightgray'])
    plt.title(title, fontsize=14)
    plt.ylabel('Valore Medio Normalizzato')
    plt.xlabel('Statistica')
    plt.legend(title=None)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_OUTPUT_PATH, save_name))
    plt.show()

# 4. Generazione dei grafici per ogni cluster
plot_cluster_comparison(0, ['pts_per_36_min', 'ast_per_36_min'], cluster_profiles_df, overall_averages, '#1f77b4', 'slide_11_cluster_0.png', 'Cluster 0 (All-Around Stars): Punti e Assist')
plot_cluster_comparison(1, ['trb_per_36_min', 'blk_per_36_min'], cluster_profiles_df, overall_averages, '#2ca02c', 'slide_12_cluster_1.png', 'Cluster 1 (Specialisti Difensivi): Rimbalzi e Stoppate')
plot_cluster_comparison(2, ['ast_per_36_min', 'tov_per_36_min'], cluster_profiles_df, overall_averages, '#d62728', 'slide_13_cluster_2.png', 'Cluster 2 (Playmaker Puri): Assist e Palle Perse')
plot_cluster_comparison(3, ['pts_per_36_min', 'trb_per_36_min', 'ast_per_36_min'], cluster_profiles_df, overall_averages, '#ffd700', 'slide_14_cluster_3.png', 'Cluster 3 (Giocatori di Ruolo): Contributo Generale')
plot_cluster_comparison(4, ['tov_per_36_min'], cluster_profiles_df, overall_averages, '#ff7f0e', 'slide_15_cluster_4.png', 'Cluster 4 (Controllo Rischio): Palle Perse')
plot_cluster_comparison(5, ['ts_pct_calc', 'pts_per_36_min'], cluster_profiles_df, overall_averages, '#9467bd', 'slide_16_cluster_5.png', 'Cluster 5 (Marcatori Efficienti): Efficienza e Punti')

### Conclusione

Tutti i grafici necessari per la presentazione sono stati generati e salvati nella cartella `reports/figures`. Sono pronti per essere importati nelle slide.

In [ ]:
# Termina la sessione Spark per liberare le risorse
spark.stop()